<a href="https://colab.research.google.com/github/sonawanekavita01/Real-Time-Industry-Insight-Strategic-Intelligence-System/blob/main/Real_Time_Industry_Insight_%26_Strategic_Intelligence_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Real-Time Industry Insight & Strategic Intelligence System
---

**Milestone 1: Introduction & Initial Training (Google Colab)**

**Goal**

Collect real-time company/industry data from open sources ( Google News, Wikipedia),preprocess it, perform sentiment analysis, and visualize insights.

In [31]:
# =========================================================
# 📊 AI-Powered Company Insight Dashboard
# =========================================================

!pip install pandas matplotlib seaborn pytrends wikipedia-api newsapi-python alpha_vantage textblob plotly

# ----------------- IMPORTS -----------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pytrends.request import TrendReq
import wikipedia
from newsapi import NewsApiClient
from alpha_vantage.timeseries import TimeSeries
from textblob import TextBlob
import plotly.express as px
import plotly.graph_objects as go
import datetime as dt

# ----------------- USER INPUT -----------------
user_input = input("Enter one or more company names separated by commas: ")
companies = [c.strip() for c in user_input.split(",") if c.strip()]

# =========================================================
# 1️⃣ WIKIPEDIA SUMMARIES
# =========================================================
wikipedia.set_lang("en")
wiki_data = []

for c in companies:
    try:
        search_results = wikipedia.search(c)
        if not search_results:
            wiki_data.append({"Company": c, "Summary": "No relevant Wikipedia page found."})
            print(f"❌ No page found for '{c}'.")
            continue

        page_title = search_results[0]
        page = wikipedia.page(page_title, auto_suggest=False)
        summary = page.summary[:800]
        wiki_data.append({"Company": c, "Summary": summary})
        print(f"✅ Summary fetched for '{c}' ({page_title})")

    except wikipedia.DisambiguationError as e:
        wiki_data.append({"Company": c, "Summary": f"Multiple results found: {e.options[:5]}."})
    except wikipedia.PageError:
        wiki_data.append({"Company": c, "Summary": "Page not found on Wikipedia."})

df_wiki = pd.DataFrame(wiki_data)
df_wiki.to_csv("company_wikipedia.csv", index=False)
display(df_wiki)

# =========================================================
# 2️⃣ MOCK GOOGLE TRENDS DATA
# =========================================================
dates = pd.date_range(end=dt.date.today(), periods=90)
mock_trends = pd.DataFrame({
    "date": np.tile(dates, len(companies)),
    "Interest": np.random.randint(30, 100, 90 * len(companies)),
    "Company": np.repeat(companies, 90)
})
display(mock_trends.head())

fig = px.line(
    mock_trends,
    x='date',
    y='Interest',
    color='Company',
    title='📈 Google Search Interest (3 Months)',
    markers=True,
    template='plotly_white'
)
fig.show()

# =========================================================
# 3️⃣ FETCH NEWS ARTICLES (NewsAPI)
# =========================================================
newsapi = NewsApiClient(api_key="fe2dd77562c94dd2aa1f590c2372050e")

news_data = []
for c in companies:
    headlines = newsapi.get_everything(q=c, language='en', sort_by='publishedAt', page_size=30)
    for art in headlines['articles']:
        news_data.append({
            "Company": c,
            "Title": art['title'],
            "PublishedAt": art['publishedAt'],
            "Source": art['source']['name']
        })

df_news = pd.DataFrame(news_data)
df_news['PublishedAt'] = pd.to_datetime(df_news['PublishedAt'])
df_news.to_csv("company_news.csv", index=False)
display(df_news.head())

# =========================================================
# 4️⃣ INTERACTIVE NEWS CHARTS
# =========================================================

# --- ✅ Fixed News Count per Company ---
news_count = df_news['Company'].value_counts().reset_index()
news_count.columns = ['Company', 'Count']  # Rename columns properly

fig1 = px.bar(
    news_count,
    x='Count',
    y='Company',
    orientation='h',
    text='Count',
    color='Company',
    title="📰 Recent News Mentions per Company",
    template='plotly_white',
    color_discrete_sequence=px.colors.qualitative.Set2
)

fig1.update_traces(textposition='outside')
fig1.update_layout(
    xaxis_title="News Count",
    yaxis_title="Company",
    showlegend=False,
    height=400,
    width=700
)

fig1.show()


# =========================================================
# 5️⃣ SENTIMENT ANALYSIS (TextBlob)
# =========================================================
def get_sentiment(text):
    polarity = TextBlob(text).sentiment.polarity
    if polarity > 0.1:
        return "Positive"
    elif polarity < -0.1:
        return "Negative"
    else:
        return "Neutral"

df_news["Sentiment"] = df_news["Title"].apply(get_sentiment)
sentiment_summary = df_news.groupby(["Company", "Sentiment"]).size().reset_index(name="Count")
display(sentiment_summary)

fig3 = px.bar(
    sentiment_summary,
    x="Company",
    y="Count",
    color="Sentiment",
    barmode="group",
    title="💬 Sentiment Analysis of Company News",
    template="plotly_white",
    color_discrete_map={"Positive": "green", "Neutral": "gray", "Negative": "red"},
    text="Count"
)
fig3.update_traces(textposition="outside")
fig3.update_layout(xaxis_title="Company", yaxis_title="Article Count", legend_title="Sentiment")
fig3.show()

# =========================================================
# 6️⃣ STOCK PRICE DATA (Alpha Vantage)
# =========================================================

from alpha_vantage.timeseries import TimeSeries
import plotly.express as px
import time

ALPHA_VANTAGE_API_KEY = "EIEDJPKIYK4PRPQR"
ts = TimeSeries(key=ALPHA_VANTAGE_API_KEY, output_format='pandas')

# 🔧 Map company names to valid Alpha Vantage symbols
symbol_map = {
    "Tata": "TATAMOTORS.BSE",
    "Tata Motors": "TATAMOTORS.BSE",
    "Reliance": "RELIANCE.BSE",
    "Reliance Industries": "RELIANCE.BSE",
    "Wipro": "WIPRO.BSE",
    "Infosys": "INFY.BSE",
    "HDFC Bank": "HDFCBANK.BSE"
}

for c in companies:
    symbol = symbol_map.get(c, None)  # ✅ Correct usage of get()
    if not symbol:
        print(f"⚠️ No symbol found for '{c}'. Please add it to symbol_map.")
        continue

    try:
        # Sleep to respect API rate limits
        time.sleep(15)

        data, meta_data = ts.get_daily(symbol=symbol, outputsize='compact')
        data.reset_index(inplace=True)

        fig4 = px.line(
            data,
            x='date',
            y='4. close',
            title=f"📊 Daily Closing Price - {c} ({symbol})",
            template='plotly_white'
        )
        fig4.update_layout(
            xaxis_title="Date",
            yaxis_title="Close Price (INR)",
            hovermode="x unified"
        )
        fig4.show()

    except Exception as e:
        print(f"⚠️ Could not fetch stock data for {c} ({symbol}): {e}")


# =========================================================
# 🎯 FINAL OUTPUT
# =========================================================
print("\n✅ Dashboard execution completed successfully!")
print("📂 Files generated: company_wikipedia.csv, company_news.csv")
print("📊 Visuals displayed interactively above.")

# =========================================================
# 🧭 MILESTONE 1 – PROJECT CONCLUSION SUMMARY
# =========================================================

from IPython.display import Markdown

milestone_summary = """
# 🏁 **Milestone 1 — AI-Powered Company Insight Dashboard**

### 🎯 **Objective**
The aim of Milestone 1 was to establish the foundational framework for an *AI-driven company insight system* that integrates multiple data sources to generate meaningful, real-time business intelligence.

---

### 🧩 **Key Achievements in Milestone 1**
1. **Wikipedia Integration:**
   Implemented automated company summary retrieval to provide background context.

2. **Google Trends Simulation:**
   Created a mock 90-day search-interest dataset to visualize comparative public attention across companies.

3. **NewsAPI Integration:**
   Gathered recent news headlines for each company to assess media presence and market coverage.

4. **Sentiment Analysis (TextBlob):**
   Applied NLP-based polarity scoring to categorize news as *Positive*, *Negative*, or *Neutral*.

5. **Stock Data Integration (Alpha Vantage):**
   Connected financial APIs to extract and visualize daily stock-price movements.

6. **Interactive Visualization (Plotly):**
   Developed dynamic, user-friendly dashboards for trend, sentiment, and stock insights.

---

### 💡 **Insights from Milestone 1**
- Public interest and sentiment vary significantly across sectors and time periods.
- Combining qualitative (news sentiment) and quantitative (stock performance) data enhances decision-making.
- Interactive dashboards enable intuitive pattern exploration and comparative company analysis.

---

### 🛠️ **Tools & Technologies**
**APIs:** Wikipedia | NewsAPI | Alpha Vantage
**Libraries:** Pandas | NumPy | Plotly | TextBlob | Seaborn | Matplotlib
**Environment:** Google Colab

---

### 🚀 **Conclusion**
Milestone 1 successfully established the **core data pipeline and visualization layer** of the AI-Powered Company Insight Dashboard.
It demonstrates how data from diverse online sources can be unified into a single analytical platform.

---

### 🔭 **Next Steps (Milestone 2 Preview)**
- Integrate **real Google Trends API** instead of mock data.
- Implement **automated daily data refresh and storage**.
- Extend sentiment analysis with **advanced NLP models (e.g., BERT)**.
- Build a **Streamlit or Power BI interface** for broader usability.

---

**✅ Milestone 1 Outcome:**
A functional and interactive analytical dashboard prototype that consolidates company insights, sentiment, and financial trends — forming the groundwork for deeper AI-based intelligence in future milestones.
"""

display(Markdown(milestone_summary))




Enter one or more company names separated by commas: Tata,Infosys,Wipro
✅ Summary fetched for 'Tata' (Tata Group)
✅ Summary fetched for 'Infosys' (Infosys)
✅ Summary fetched for 'Wipro' (Wipro)


,Company,Summary
0,Tata,The Tata Group () is an Indian multinational g...
1,Infosys,Infosys Limited is an Indian multinational tec...
2,Wipro,Wipro Limited ( ) is an Indian multinational t...


,date,Interest,Company
0,2025-08-07,32,Tata
1,2025-08-08,54,Tata
2,2025-08-09,47,Tata
3,2025-08-10,44,Tata
4,2025-08-11,80,Tata


,Company,Title,PublishedAt,Source
0,Tata,"Titan Q2 Results: Profit Surges 59% To Rs 1,12...",2025-11-03 14:15:03+00:00,Ndtvprofit.com
1,Tata,Sequestration and suppressed synthesis of onco...,2025-11-03 14:00:00+00:00,Plos.org
2,Tata,Cipla to buy Inzpera Healthsciences for ₹110 c...,2025-11-03 13:56:39+00:00,BusinessLine
3,Tata,Cipla to buy Inzpera Healthsciences for Rs 110...,2025-11-03 13:56:39+00:00,BusinessLine
4,Tata,Daimler India Commercial Vehicle eyes €400 mil...,2025-11-03 13:51:37+00:00,BusinessLine


,Company,Sentiment,Count
0,Infosys,Negative,3
1,Infosys,Neutral,20
2,Infosys,Positive,7
3,Tata,Negative,3
4,Tata,Neutral,19
5,Tata,Positive,8
6,Wipro,Negative,2
7,Wipro,Neutral,24
8,Wipro,Positive,4



✅ Dashboard execution completed successfully!
📂 Files generated: company_wikipedia.csv, company_news.csv
📊 Visuals displayed interactively above.



# 🏁 **Milestone 1 — AI-Powered Company Insight Dashboard**

### 🎯 **Objective**
The aim of Milestone 1 was to establish the foundational framework for an *AI-driven company insight system* that integrates multiple data sources to generate meaningful, real-time business intelligence.

---

### 🧩 **Key Achievements in Milestone 1**
1. **Wikipedia Integration:**  
   Implemented automated company summary retrieval to provide background context.

2. **Google Trends Simulation:**  
   Created a mock 90-day search-interest dataset to visualize comparative public attention across companies.

3. **NewsAPI Integration:**  
   Gathered recent news headlines for each company to assess media presence and market coverage.

4. **Sentiment Analysis (TextBlob):**  
   Applied NLP-based polarity scoring to categorize news as *Positive*, *Negative*, or *Neutral*.

5. **Stock Data Integration (Alpha Vantage):**  
   Connected financial APIs to extract and visualize daily stock-price movements.

6. **Interactive Visualization (Plotly):**  
   Developed dynamic, user-friendly dashboards for trend, sentiment, and stock insights.

---

### 💡 **Insights from Milestone 1**
- Public interest and sentiment vary significantly across sectors and time periods.  
- Combining qualitative (news sentiment) and quantitative (stock performance) data enhances decision-making.  
- Interactive dashboards enable intuitive pattern exploration and comparative company analysis.

---

### 🛠️ **Tools & Technologies**
**APIs:** Wikipedia | NewsAPI | Alpha Vantage  
**Libraries:** Pandas | NumPy | Plotly | TextBlob | Seaborn | Matplotlib  
**Environment:** Google Colab  

---

### 🚀 **Conclusion**
Milestone 1 successfully established the **core data pipeline and visualization layer** of the AI-Powered Company Insight Dashboard.  
It demonstrates how data from diverse online sources can be unified into a single analytical platform.

---

### 🔭 **Next Steps (Milestone 2 Preview)**
- Integrate **real Google Trends API** instead of mock data.  
- Implement **automated daily data refresh and storage**.  
- Extend sentiment analysis with **advanced NLP models (e.g., BERT)**.  
- Build a **Streamlit or Power BI interface** for broader usability.

---

**✅ Milestone 1 Outcome:**  
A functional and interactive analytical dashboard prototype that consolidates company insights, sentiment, and financial trends — forming the groundwork for deeper AI-based intelligence in future milestones.


In [29]:
from google.colab import files
files.download("company_wikipedia.csv")
files.download("company_news.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>